In [1]:
import pandas as pd
import numpy as np
from apyori import apriori  

In [2]:
df = pd.read_csv("../data/location1_cleaned.csv")
df_prods = pd.read_csv("../data_engineering/product_image_mapping.csv")

In [3]:
del df['product_sku']
del df['display_description']
del df['location_name']

In [4]:
df.head(10)

,order_id,order_time(UTC),items_id,items_name,item_price,quantity,served_for,item_category_type
0,841222,2018-07-31 19:02:57,2255,Bud Light,7.50,1,Lunch,Alcoholic Beverage
1,841222,2018-07-31 19:02:57,3038,"Shrimp, Fish and Chicken Platter",15.99,1,Lunch,Entree
2,841222,2018-07-31 19:02:57,3038,"Shrimp, Fish and Chicken Platter",15.99,1,Lunch,Entree
3,841222,2018-07-31 19:02:57,2252,Assorted Fountain Beverages - Regular,3.29,1,Lunch,Beverage
4,841222,2018-07-31 19:02:57,3036,Shrimp Platter,12.49,1,Lunch,Entree
5,841222,2018-07-31 19:02:57,3036,Shrimp Platter,12.49,1,Lunch,Entree
6,841231,2018-07-31 19:05:20,3236,Toy Story Land Cupcake,4.99,1,Lunch,Dessert
7,841231,2018-07-31 19:05:20,3236,Toy Story Land Cupcake,4.99,1,Lunch,Dessert
8,841237,2018-07-31 19:08:43,2716,Chicken Breast Strips,9.49,1,Lunch,Entree
9,841237,2018-07-31 19:08:43,2253,Assorted Fountain Beverages - Large,3.69,1,Lunch,Beverage


In [45]:
# make large and small fountain drinks be the same thing
df.loc[ df['items_id'] == 2253,'items_id' ] = 2252 
# combine the same cupcake
df.loc[ df['items_id'] == 3411] = 3236
# combine the same Olaf cupcake
df.loc[ df['items_id'] == 3410] = 2245
# combine mac and chees
df.loc[ df['items_id'] == 3274] = 3276

In [46]:
#Categories:
cats = list(set(list(df_prods['category'])))
cats


['Entree', 'Alcoholic Beverage', 'Dessert', 'Beverage', 'Kids', 'Side']

In [47]:
# most popular items
ydf = (df.groupby('items_id').count()['order_id']).reset_index(drop=False)
ydf.rename(columns={'order_id':'count'},inplace=True)
df_prods.rename(columns={'id':'items_id'},inplace=True)
zdf = pd.merge(ydf,df_prods,on='items_id',how='right')
zdf = zdf[ zdf['location'] != "ABC Commissary"]
zdf = zdf.sort_values(['count'],ascending=False)
zdf.head(5)

,items_id,count,name,display_desc,category,location,img
19,2252,12597.0,Assorted Fountain Beverages - Regular,Assorted Coca-Cola® Offerings,Beverage,Columbia Harbor House,bev_fountain_beverages.jpeg
9,2237,4800.0,Mediterranean Salad with Chicken,tossed in a Mediterranean Vinaigrette and serv...,Entree,Columbia Harbor House,salad_mediterranean_salad_chicken.png
32,3038,4432.0,"Shrimp, Fish and Chicken Platter",served with Steak Fries and Coleslaw,Entree,Columbia Harbor House,fried_shrimp_fish_chicken.png
28,2716,4382.0,Chicken Breast Strips,"served with your choice of fresh Green Beans, ...",Entree,Columbia Harbor House,fried_chicken-strips.png
3,2210,4067.0,1/3 lb Angus Bacon Cheeseburger,"served with your choice of fresh Green Beans, ...",Entree,Columbia Harbor House,burger_angus_bacon_cheeseburger.jpeg


In [55]:
entrees = zdf[ zdf['category'] == "Entree" ]

In [49]:
bev = zdf[ zdf['category'] == "Beverage" ]

In [50]:
alc = zdf[ zdf['category'] == "Alcoholic Beverage" ]

In [51]:
sides = zdf[ zdf['category'] == "Side" ]

In [52]:
desserts = zdf[ zdf['category'] == "Dessert" ]

In [65]:
kids = zdf[ zdf['category'] == "Kids" ]

In [66]:
import collections
most_popular = collections.OrderedDict()

In [67]:
most_popular['entree'] = list(entrees.iloc[0:3][ 'items_id' ])
most_popular['side'] = list(sides.iloc[0:3][ 'items_id' ])
most_popular['dessert'] = list(desserts.iloc[0:3][ 'items_id' ])
most_popular['beverage'] = list(bev.iloc[0:3][ 'items_id' ])
most_popular['alcohol'] = list(alc.iloc[0:3]['items_id'])
most_popular['kids'] = list(kids.iloc[0:3]['items_id'])

In [68]:
import json
with open("../data/most_popular.json","w") as outfile:
    outfile.write(json.dumps(most_popular))